In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time
from tqdm import trange, tqdm

from trainer import Trainer, generate_data_iter
import utils
import attack
import cam

In [2]:
model_mode, dataset, seed, cuda = 'resnet18', 'CIFAR10', 0, 2
target_layer = 'layer4' if model_mode == 'resnet18' else 'blocks'
fig_num = 400
metrics = {}
cam_metrics = {}

trainer = Trainer(
    model_mode = model_mode,
    dataset = dataset,
    bs = 128,
    lr = 0.01,
    seed = seed,
    cuda = cuda,
    use_lr_sche = True,
    use_wandb = False,
)
trainer.load()
scorecam = cam.CAM(trainer.model, dataset, target_layer, 'fc', cuda = cuda)

test_iter = generate_data_iter(dataset, batch_size = fig_num, mode = 'test')
imgs, labels = next(iter(test_iter))
raw_cam_nps, raw_preds, _, __ = scorecam(imgs, metric = False)
suc_indices = raw_preds == labels
print(f'Acc on fig_num Images: {suc_indices.sum() / len(labels) * 100:4.2f}%')

suc_imgs, suc_labels = imgs[suc_indices], labels[suc_indices]
suc_cam_nps, suc_preds, suc_probs, suc_cam_metrics = scorecam(suc_imgs, metric = True)
suc_saliency_maps, _, __ = scorecam.generate_saliency_map(scorecam.tfm(suc_imgs))

cam_metrics['Raw'] = {
    'Average Incr': suc_cam_metrics['Average Incr'], 'Average Drop': suc_cam_metrics['Average Drop'],
    'Insertion':  suc_cam_metrics['Insertion'], 'Deletion': suc_cam_metrics['Deletion'],
}
print(cam_metrics['Raw'])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Acc on fig_num Images: 94.75%


KeyError: 'Raw'

### FGSM

In [3]:
att_method = attack.FGSM(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], suc_labels[i])
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

fgsm_cams, fgsm_preds, fgsm_probs, fgsm_cam_metrics = scorecam(attacks, metric = True)
fgsm_saliency_maps, _, __ = scorecam.generate_saliency_map(scorecam.tfm(suc_imgs))
success_rate = (fgsm_preds != suc_labels).sum() / len(fgsm_preds)

print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}
cam_metrics[att_name] = {
    'Average Incr': fgsm_cam_metrics['Average Incr'], 'Average Drop': fgsm_cam_metrics['Average Drop'],
    'Insertion':  fgsm_cam_metrics['Insertion'], 'Deletion': fgsm_cam_metrics['Deletion'],
}
print(cam_metrics[att_name])

100%|██████████| 379/379 [00:09<00:00, 38.09it/s]


FGSM:	Success Rate: 83.64%,  Norm: 1.958e-01,  Avg Time: 0.025s


KeyError: 'Raw'

### L-BFGS

In [3]:
att_method = attack.LBFGS(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
target_clses = (suc_labels + 1) % 10
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], target_clses[i])
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

[378/379]: 100%|██████████| 379/379 [16:50<00:00,  2.67s/it]

LBFGS:	Success Rate: 55.41%,  Avg Time: 2.525s


### FGSM

100%|██████████| 379/379 [00:03<00:00, 106.38it/s]


FGSM:	Success Rate: 83.64%,  Norm: 1.958e-01,  Avg Time: 0.009s


### FGM

In [5]:
att_method = attack.FGM(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], suc_labels[i], eps = 10)
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:02<00:00, 128.30it/s]


FGM:	Success Rate: 82.85%,  Norm: 3.115e-01,  Avg Time: 0.007s


### I-FGSM

In [6]:
att_method = attack.IFGSM(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], suc_labels[i])
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:14<00:00, 26.07it/s]


IFGSM:	Success Rate: 100.00%,  Norm: 2.316e-01,  Avg Time: 0.036s


### PGD

In [3]:
att_method = attack.IFGSM(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], suc_labels[i])
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:25<00:00, 14.87it/s]


IFGSM:	Success Rate: 100.00%,  Norm: 2.318e-01,  Avg Time: 0.064s


### MIFGSM

In [7]:
att_method = attack.MIFGSM(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], suc_labels[i])
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:14<00:00, 26.06it/s]


MIFGSM:	Success Rate: 100.00%,  Norm: 2.402e-01,  Avg Time: 0.036s


### NIFGSM

In [8]:
att_method = attack.NIFGSM(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], suc_labels[i])
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:14<00:00, 26.59it/s]


NIFGSM:	Success Rate: 100.00%,  Norm: 2.403e-01,  Avg Time: 0.036s


### Step-LL

In [9]:
att_method = attack.StepLL(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], 10, eps = 0.05)
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:04<00:00, 88.59it/s]


StepLL:	Success Rate: 80.21%,  Norm: 9.893e-02,  Avg Time: 0.011s


## IterLL

In [10]:
att_method = attack.IterLL(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], 10)
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:15<00:00, 24.52it/s]


IterLL:	Success Rate: 100.00%,  Norm: 2.307e-01,  Avg Time: 0.039s


### Deep Fool

In [11]:
att_method = attack.DeepFool(trainer.model)
att_name = att_method.__class__.__name__
attacks = torch.zeros_like(suc_imgs)
start = time.time()
with trange(len(suc_imgs)) as t:
    for i in t:
        attacks[i] = att_method(suc_imgs[i], 10)
final = time.time()

deltas = attacks - suc_imgs
norm = torch.mean(
    torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
        / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
)

att_preds, att_probs = trainer.model_predict(attacks)
success_rate = (att_preds != suc_labels).sum() / len(att_preds)
print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
metrics[att_name] = {
    'success_rate': (success_rate * 100).item(),
    'norm': norm.item(),
    'time': final - start
}

100%|██████████| 379/379 [00:21<00:00, 17.77it/s]

DeepFool:	Success Rate: 90.24%,  Norm: 2.793e-01,  Avg Time: 0.053s


### One Pixel

In [12]:
# att_method = attack.OnePixel(trainer.model)
# att_name = att_method.__class__.__name__
# attacks = torch.zeros_like(suc_imgs)
# start = time.time()
# with trange(len(suc_imgs)) as t:
#     for i in t:
#         attacks[i] = att_method(suc_imgs[i], suc_labels[i], 100, 400)
# final = time.time()

# deltas = attacks - suc_imgs
# norm = torch.mean(
#     torch.linalg.norm(deltas.reshape(len(deltas), -1), dim = 1) \
#         / torch.linalg.norm(suc_imgs.reshape(len(deltas), -1), dim = 1)
# )

# att_preds, att_probs = trainer.model_predict(attacks)
# success_rate = (att_preds != suc_labels).sum() / len(att_preds)
# print(f'{att_name}:\tSuccess Rate: {success_rate * 100:4.2f}%,  Norm: {norm:.3e},  Avg Time: {(final - start)/fig_num:.3f}s')
# metrics[att_name] = {
#     'success_rate': success_rate * 100,
#     'norm': norm,
#     'time': final - start
# }

In [13]:
metrics_df = pd.DataFrame(metrics)

In [14]:
metrics_df

,FGSM,FGM,IFGSM,MIFGSM,NIFGSM,StepLL,IterLL,DeepFool
success_rate,83.641159,82.849602,100.000000,100.000000,100.000000,80.211082,100.000000,90.237465
norm,0.195761,0.311550,0.231609,0.240249,0.240260,0.098932,0.230711,0.279301
time,3.564740,2.955983,14.544076,14.546105,14.254607,4.280385,15.462418,21.326583


In [15]:
metrics_df.to_csv('./results/attack_metrics.csv')